In [122]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

In [119]:
def get_search_results(request_list, pages):
  df_articles = pd.DataFrame()
  URL = 'https://habr.com'
  search_url = URL + '/ru/search/page'

  for request in request_list:
    params = {
        'q': request,
        'target_type': 'posts',
        'order': 'relevance'
    }

    for page in range(pages):
      link = '' # Для проверки наличия ссылки
      search_url_with_page = search_url + str(page + 1) + '/'
      res = requests.get(search_url_with_page, params=params)
      time.sleep(0.1)
      soup = BeautifulSoup(res.text)
      articles = soup.find_all('article', class_='tm-articles-list__item')

      for article in articles:
        if article.find('div', class_='tm-article-snippet'):
          date = article.find('span', class_='tm-article-datetime-published').find('time').text
          title = article.find('h2', class_='tm-title').find('span').text
          link = article.find('h2', class_='tm-title').find('a').get('href')
        elif article.find('div', class_='tm-megapost-snippet'):
          date = article.find('time', class_='tm-megapost-snippet__datetime-published').text
          title = article.find('h2', class_='tm-megapost-snippet__title').text
          link = article.find('a', class_='tm-megapost-snippet__link').get('href')

        if link != '':
          request_text = requests.get(URL + link).text
          soup_text = BeautifulSoup(request_text)
          time.sleep(0.1)

          # Проверка на несуществующую страницу статьи и отсутствие блога компании
          if soup_text.find('h1', class_='tm-error-message__title') or soup_text.find('div', class_='tm-expired-company'):
            content = ''
            likes = article.find('span', class_='tm-votes-meter__value').text
          else:
            likes = soup_text.find('span', class_='tm-votes-meter__value').text
            if article.find('div', class_='tm-article-snippet'):
              content = soup_text.find('div', class_='article-formatted-body').text.strip()
            elif article.find('div', class_='tm-megapost-snippet'):
              content = soup_text.find('div', class_='t-records').text.strip()

        row = {'date': date, 'title': title, 'link': URL + link, 'text': content, 'likes': likes}
        df_articles = pd.concat([df_articles, pd.DataFrame([row])])

  return df_articles.reset_index(drop=True).drop_duplicates()

In [121]:
result = get_search_results(['python', 'анализ данных', 'python'], 3)
result

,date,title,link,text,likes
0,3 мар 2020 в 13:22,В начале этого года Python сместил Java и стал...,https://habr.com/ru/companies/itsumma/news/490...,"Согласно отчету RedMonk за январь 2020 года, P...",+31
1,23 июн в 16:17,Как можно компилировать типизированный Python,https://habr.com/ru/companies/piter/articles/7...,"Прошло уже целых 9 лет с тех пор, как состоялс...",+11
2,20 июн в 11:16,"Типизация в Python. Работа с Mypy, PyCharm и S...",https://habr.com/ru/companies/selectel/article...,"Привет, Хабр! Мы в Selectel много программируе...",+12
3,31 окт 2019 в 10:02,Создатель Python Гвидо ван Россум ушел из Drop...,https://habr.com/ru/news/473926/,Создатель языка программирования Python Гвидо ...,+23
4,31 мая в 09:17,25 бесплатных курсов по Python 2023 года,https://habr.com/ru/articles/738438/,"Тема обучения «Питону» — вечнозелёная, посколь...",+7
...,...,...,...,...,...
115,17 окт 2018 в 08:15,Функциональность SAP HANA как базы данных для ...,https://habr.com/ru/companies/sap/articles/426...,Мы продолжаем цикл статей о SAP HANA Data Mana...,+3
116,16 мар 2020 в 10:56,Какой язык выбрать для работы с данными — R ил...,https://habr.com/ru/articles/475210/,По запросу R или Python в интернете вы найдёте...,+19
117,26 ноя 2021 в 21:20,Как Business Intelligence «купается» в озёрах ...,https://habr.com/ru/companies/fsight/articles/...,Часть 1. Технология гетерогенных ROLAP-кубовВс...,+3
118,6 сен 2017 в 21:17,Какой язык программирования выбрать для работы...,https://habr.com/ru/articles/337330/,У начинающего специалиста по данным (data scie...,+7
